In [1]:
import teneva
import pysabr
import numpy as np
import time
import pandas as pd

In [2]:
Ss = np.linspace(0.005, 0.07, num=3)
Ks = np.linspace(0.005, 0.07, num=3)
Ts = np.linspace(0.5, 15., num=3)
V_atms = np.linspace(0.1, 1.5, num=3) / 100
betas = np.linspace(0.1, 0.7, num=3)
rhos = np.linspace(-0.4, 0.4, num=3)
volvols = np.linspace(0.0001, 0.5, num=3)
displacements = np.linspace(0.0, 0.03, num=5)

In [3]:
start = time.time()
sabr_processes = [pysabr.Hagan2002LognormalSABR(S, d, T, V_atm, beta, rho, volvol) \
                  for S in Ss \
                  for T in Ts \
                  for V_atm in V_atms \
                  for beta in betas \
                  for rho in rhos \
                  for volvol in volvols \
                  for d in displacements
                 ]

vols = np.reshape([sabr.lognormal_vol(k) for sabr in sabr_processes for k in Ks], \
                   (Ss.shape[0], Ts.shape[0], V_atms.shape[0], betas.shape[0], rhos.shape[0], \
                    volvols.shape[0], displacements.shape[0], Ks.shape[0])\
                 )

end = time.time()
print(f'compute vols time: {end - start}')

# res = np.zeros((Ss.shape[0],\
#                 Ts.shape[0],\
#                 V_atms.shape[0],\
#                 betas.shape[0],\
#                 rhos.shape[0],\
#                 volvols.shape[0],\
#                 displacements.shape[0],\
#                 Ks.shape[0]))

# for S in Ss.shape[0]:
#     for T in Ts.shape[0]:
#         for V_atm in V_atms.shape[0]:
#             for beta in betas.shape[0]:
#                 for rho in rhos.shape[0]:
#                     for volvol in volvols.shape[0]:
#                         for d in displacements.shape[0]:
#                             sabr_ln = Hagan2002LognormalSABR(S, d, T, V_atm, beta, rho, volvol)
#                             res[S, T, V_atm, beta, rho, volvol, displacement] = [sabr_ln.lognormal_vol(k/100) for k in strikes]

compute vols time: 64.58814454078674


In [4]:
print(vols.shape)

(3, 3, 3, 3, 3, 3, 5, 3)


In [5]:
pd.DataFrame(vols.reshape((-1,))).describe()

,0
count,10935.000000
mean,0.352684
std,0.606969
min,0.010000
25%,0.086696
50%,0.193700
75%,0.341945
max,4.505684


In [6]:
data_path = "data/"

In [7]:
# np.save(data_path + "sample_lognormal_vol.npy", vols)